# <font color='green'>Dynamics of recurrent network of excitatory and inhbitory neurons</font>

## <font color='green'>Aims of the tutorial </font>
- Simulate a randomly connected network of excitatory and inhibitory neurons
- Study the effect of change in recurrent inhibition on the spiking activity, in particular on spike synchronization and spike pattern irregularity
## <font color='green'>Learning outcomes</font>
- An understanding of how recurrent inhibition affectx spike synchronization and spike pattern irregularity
- A basic understanding of simulation of networks in NEST
- An understanding of descriptors of spiking activity of neurons in a network
## <font color='green'>What will you do</font>
- Execute the Python code and display the results and describe your observations
- Write small code snippetes to perform certain simulations or calculations


## <font color='green'>Random network model of the neocortex</font>
At the simplest level every network in the brain is made of either inhibitory neurons or both excitatory and inhibitory neurons. In the neocortex most networks are composed of 70-80% excitatory neurons and 20-30 inhibitory neurons. Of course these neurons can be classified further in different categories. 

When we think of a network, we need to define the connectivity between nodes (here neurons). Experimental data suggests that in the neocortex average connection probabilty measured over 100$\mu$m of distance is sparse (about 10%). And at this scale neurons are more or less randomly connected. However, if we go beyond 100$\mu$m the connection probability start to decay as a function of distance between neurons. And beyond 1cm spatial distance connectivity has very different rules.

## Assumptions
In this tutorial we will consider one of the simplest model of the neocortical tissue. We will assume that 
- The network is composed of one type of excitatory and one type of inhibitory neurons
- There are 80% excitatory neurons and 20% inhibitory neurons
- All neurons have identical properties and are  modelled as integrate-fire-neurons
- Neurons are connected in a random fashion with 10% connection probability
- All the synapses are current-based and synaptic integration is linear
- Neurons are driven by Poisson type spike trains


## You might wonder what can we learn from such a simple model? 
This model is sufficient to understand the emergence of synchonization, oscillation and stability of network activity dynamics. Synchrony and oscillations are ubiqutious properties of brain activity and play important role in encoding and transfer of information in the brain. Moreover, a number of brain diseases are manifested in terms of changes in synchrony and oscillations. For instance. Parkinson's disease is a diseases of increased oscillations. 

These modellled have successfully and correctly revealed the role of balancing the excitation and inhibition in a network in order to control not only firing rates but also synchrony and oscillations. 


<img src="./Figures/EI-scheme.png">

- $J_{EE}$ connection weight from an excitatory to an exhibitory neuron, $J_{EI}$ connection weight from an inhibitory to an exhibitory neuron, $J_{IE}$ connection weight from an excitatory to an inhibitory neuron, $J_{EI}$ connection weight from an inhibitory to an inhibitory neuron.
- We will assume $J_{EE}$ = $J_{IE}$ and $J_{II}$ = $J_{EI}$ (In the code below  $J_{EE} = J_{ex}$ and  $J_{II} = J_{in}$)
- We will also assume $J_{EE}$ = $gJ_{II}$. That is $g$ is the ratio of excitation and inhibition
- In this tutorial we will keep the external input to the two populations same $E_E = E_I$ (in the code below these parameters are callled eta_ex and eta_in)
- This means $E_E$ and $g$ are the key parameters that you will vary in this tutorial and investigate how these two parameters affect the synchrony

## Reference 
- Brunel N, Dynamics of Sparsely Connected Networks of Excitatory and Inhibitory Spiking Neurons, Journal of Computational Neuroscience 8, 183-208 (2000).
- Kumar et al. High conductance state of cortial netwokrs. Neural Computations. 20(1):1-43.(2008)
- Rennart et al. The asynchronous state in cortical circuits. Science  327(5965): 587–590. (2010)

# <font color=blue>Describe</font>
### When all the neurons are the same, they are connected randomly and are driven by Poisson type spike trains why do you think that there should be any synchrony or oscillations in the network at all? Or under what conditions this network will synchronize or oscillate if at all?


### Some preliminaries

In [ ]:
from scipy.optimize import fsolve
import numpy as np
from numpy import exp
import time
import matplotlib.pyplot as plt
%matplotlib inline

import nest

#### Definition of functions used in this example. 
First, define the Lambert W function. This function will be used to calibrate the synaptic weights.

In [ ]:
def LambertWm1(x):
    nest.sli_push(x)
    nest.sli_run('LambertWm1')
    y = nest.sli_pop()
    return y

def ComputePSPnorm(tauMem, CMem, tauSyn):
    a = (tauMem / tauSyn)
    b = (1.0 / tauSyn - 1.0 / tauMem)

    # time of maximum
    t_max = 1.0 / b * (-LambertWm1(-exp(-1.0 / a) / a) - 1.0 / a)

    # maximum of PSP for current of unit amplitude
    return (exp(1.0) / (tauSyn * CMem * b) *
            ((exp(-t_max / tauMem) - exp(-t_max / tauSyn)) / b -
             t_max * exp(-t_max / tauSyn)))

In [ ]:
# Simulation parameters
dt = 0.1    # the resolution in ms
simtime = 500.0  # Simulation time in ms
delay = 1.5    # synaptic delay in ms

### Network size and fraction of exc. and inh. neurons

In [ ]:
order = 1000
NE = 4 * order  # number of excitatory neurons
NI = 1 * order  # number of inhibitory neurons
N_neurons = NE + NI   # number of neurons in total
N_rec = 500      # record from 50 neurons

### Parameters describing the global synchrony and balance
-- By varying these parameters we can modulate synchrony and oscillations

In [ ]:
g = 5.0  # ratio inhibitory weight/excitatory weight
eta_ex = 2.5  # external rate relative to threshold rate --> Exc. Neurons
eta_in = 2.5  # external rate relative to threshold rate --> Inh. Neurons
epsilon = 0.1  # connection probability

### Network connectivity

In [ ]:
CE = int(epsilon * NE)  # number of excitatory synapses per neuron
CI = int(epsilon * NI)  # number of inhibitory synapses per neuron
C_tot = int(CI + CE)      # total number of synapses per neuron

### Neuron and synapse parameters

In [ ]:
tauSyn = 0.5  # synaptic time constant in ms
tauMem = 20.0  # time constant of membrane potential in ms
CMem = 250.0  # capacitance of membrane in in pF
theta = 20.0  # membrane threshold potential in mV
neuron_params = {"C_m": CMem,
                 "tau_m": tauMem,
                 "tau_syn_ex": tauSyn,
                 "tau_syn_in": tauSyn,
                 "t_ref": 2.0,
                 "E_L": 0.0,
                 "V_reset": 0.0,
                 "V_m": 0.0,
                 "V_th": theta}
J = 0.1        # postsynaptic amplitude in mV
J_unit = ComputePSPnorm(tauMem, CMem, tauSyn)
J_ex = J / J_unit  # amplitude of excitatory postsynaptic current
J_in = -g * J_ex    # amplitude of inhibitory postsynaptic current

### Inpute rate

In [ ]:
nu_th = (theta * CMem) / (J_ex * CE * np.exp(1) * tauMem * tauSyn)
nu_ex = eta_ex * nu_th # to excitatory neurons
p_rate_ex = 1000.0 * nu_ex * CE

nu_in = eta_in * nu_th # To inhibitory neurons
p_rate_in = 1000.0 * nu_in * CE

### Network buildup and default parameters
(If you change some parameter above it is useful to reset the kernel and start simultion from zero)

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({"resolution": dt, "print_time": True,
                      "overwrite_files": True})

print("Building network")

nest.SetDefaults("iaf_psc_alpha", neuron_params)
nest.SetDefaults("poisson_generator", {"rate": p_rate_ex})


### Create neurons and spike detectors

In [ ]:
nodes_ex = nest.Create("iaf_psc_alpha", NE)
nodes_in = nest.Create("iaf_psc_alpha", NI)
noise_ex = nest.Create("poisson_generator")
noise_in = nest.Create("poisson_generator")
espikes = nest.Create("spike_detector")
ispikes = nest.Create("spike_detector")
multimeter = nest.Create("multimeter")

### Change spike threshold of a few neurons 

In [ ]:
# neuron id 0 and neuron id 1 willnot spike because the spike threshold is too high 
#and we can see the free membrane potential
nest.SetStatus([nodes_ex[0]],{'V_th':1e4})
nest.SetStatus([nodes_ex[1]],{'V_th':1e4})

### Configure the spike detector

In [ ]:
nest.SetStatus(espikes, [{"label": "EInet-ex",
                          "withtime": True,
                          "withgid": True,
                          "to_file": True}])

nest.SetStatus(ispikes, [{"label": "EInet-in",
                          "withtime": True,
                          "withgid": True,
                          "to_file": True}])
nest.SetStatus(multimeter, {"label": "EInet-mem",
                            "withtime":True,
                            "to_file": True,
                            "record_from":["V_m"]})

print("Connecting devices")
# The data will be saved in a file as well as in the memory

### Synaptic weights

In [ ]:
nest.CopyModel("static_synapse", "excitatory",
               {"weight": J_ex, "delay": delay})
nest.CopyModel("static_synapse", "inhibitory",
               {"weight": J_in, "delay": delay})

### Connect the noise

In [ ]:
nest.Connect(noise_ex, nodes_ex, syn_spec="excitatory")
nest.Connect(noise_in, nodes_in, syn_spec="excitatory")

### Connect the neurons to recording devices

In [ ]:
nest.Connect(nodes_ex[:N_rec], espikes, syn_spec="excitatory")
nest.Connect(nodes_in[:N_rec], ispikes, syn_spec="excitatory")
nest.Connect(multimeter,nodes_ex[0:3])

print("Connecting network")

### Connect the network

In [ ]:
np.random.seed(1234)

sources_ex = np.random.randint(1, NE + 1, (N_neurons, CE))
sources_in = np.random.randint(NE + 1, N_neurons + 1, (N_neurons, CI))

for n in range(N_neurons):
    nest.Connect(list(sources_ex[n]), [n + 1], syn_spec="excitatory")

for n in range(N_neurons):
    nest.Connect(list(sources_in[n]), [n + 1], syn_spec="inhibitory")

endbuild = time.time()

### Simulate
(if you start the simulation without resetting the kernel the simulation clock willstart from where it was at the end of last simulation)

In [ ]:
print("Simulating")

nest.Simulate(simtime)
endsimulate = time.time()

### Read the spiking activity

In [ ]:
## Uncomment the following if you want to read the spikes from the memory 
# and comment last line which reads the data from the file

#events_ex = nest.GetStatus(espikes, "n_events")[0]
#events_in = nest.GetStatus(ispikes, "n_events")[0]

# When reading from the file
# Note that the file number depends on the number of devices in the simulation.
# In this case file number is N+3 where N is the number of total neurons
xx = np.loadtxt('EInet-ex-5003-0.gdf') 

### Read the membrane potential

In [ ]:
# When reading from the file
# Note that the file number depends on the number of devices in the simulation.
# In this case file number is N+5 where N is the number of total neurons

# In .dat file first column is neuron id, second column is time stamp and thethird column is membrane potential
mem = np.loadtxt('EInet-mem-5005-0.dat')
mem1 = mem[mem[:,0]==1,:] # neuron 1
mem2 = mem[mem[:,0]==2,:] # neuron 2
mem3 = mem[mem[:,0]==3,:] # neuron 3


### Plot the spiking activity

In [ ]:
# Plot the data 
plt.figure(figsize=(20, 10))
plt.subplot(2,1,1)
plt.plot(xx[:,1],xx[:,0],'.')
plt.ylim([0,50])
plt.xlim([0,simtime])
plt.xlabel('Time (msec)')
plt.ylabel('Neuron Id.')


plt.subplot(2,1,2)
plt.plot(mem1[:,1],mem1[:,2],label='neuron 1')
plt.plot(mem2[:,1],mem2[:,2],label='neuron 2')
plt.plot(mem3[:,1],mem3[:,2],label='neuron 3')
plt.plot([0, simtime],[20, 20],label='spike threshold')
plt.ylim([0,50])
plt.legend()
plt.xlim([0,simtime])
plt.xlabel('Time (msec)')
plt.ylabel('Membrane potential (mV)')
plt.show()

# <font color=blue>Describe</font>
#### In the plot above you see the spiking activity of 50 excitatory neurons (out of 1000). Each dot is a spike and each row is a neuron. Such plots are called raster plots.

- Did you expect the above result?
- How will you quantify these results?
- What does the membrane potential suggests when you compare the membrane potentials of the two neurons that do now spike

### Lets plot the spiking activity again with more neurons

In [ ]:
# Plot the data 
plt.figure(figsize=(20, 5))
plt.plot(xx[:,1],xx[:,0],'.')
plt.ylim([0,500])
plt.xlim([0,simtime])
plt.xlabel('Time (msec)')
plt.ylabel('Neuron Id.')

plt.show()

# <font color=blue>Describe</font>
#### In the plot above you see the spiking activity of 500 excitatory neurons (out of 1000). Each dot is a spike and each row is a neuron. Such plots are called raster plots.

- Did you expect the above result after seeing the plot above? Write your reasoning.

- Why is there a big synchronous event at the beginning? What can we do to get rid of this?
- How will you quantify the properties of the spiking activty shown in the figure above?


# <font color=green>Lets do some quantification of neuron firing rate and patterns</font>
### We can count the spikes of each neuron and estimate average firing rate and distribution of spike rates

In [ ]:
# Count rate of each neuron. For this we take advantage of the gdf file format
# In the gdf file, first column is neuron id and second column records when that neuron spiked. 
# Each spike is a single line enetry
# So to estimate the firing rate of each neuron we need to count how often a neuron appears in the first column
# First lets reject first 100 ms to get rid of the startup transient

xx = xx[xx[:,1]>100,:]
fr_ed = np.arange(1,501,1)
frh,tmp = np.histogram(xx[:,0],fr_ed)

sp_rate = 1e3*frh/(simtime-100)
mean_fr = np.mean(sp_rate) # mean firing rate
std_fr = np.std(sp_rate) # std of the firing rate


# <font color=salmon>TO DO</font>
- Plot the distribution of firing rates

### Besides counting the spikes, we can also ask how regularly neurons are spiking. A clock is something that ticks regular. So we can do neurons spike regularly like a clock.

In [ ]:
# Lets get the interspike intervals of each neuron
sel_neuron = 100
cv = np.empty(sel_neuron)
for ii in range(sel_neuron):
    x1 = xx[xx[:,0]==ii,:] # select a neuron
    if len(x1[:,1])>3:
        isi = np.diff(x1[:,1]) # get the interspike intervals
        cv[ii] = np.std(isi)/np.mean(isi)

mean_cv = np.nanmean(cv)

print(mean_cv)

# <font color=salmon>TO DO</font>
- Run a longer simulation (e.g. 5000ms) and plot the distribution of spike times of a single neurons


# <font color=green>Lets do some quantification of activity as a population</font>

### First we estimate the population firing rate

In [ ]:

bin_size = 2
ed = np.arange(100,simtime,bin_size)
hh,edx = np.histogram(xx[:,1],ed)
hh = hh/(bin_size*1e-3)/np.float(N_rec)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(2,1,1)
plt.plot(xx[:,1],xx[:,0],'.')
plt.xlim([100,simtime])
plt.ylabel('Neuron Id.')

plt.subplot(2,1,2)
plt.plot(ed[:-1],hh)
plt.xlim([100,simtime])
plt.xlabel('Time (msec)')
plt.ylabel('Firing rate')

plt.show()

# <font color=blue>Describe</font>
#### In the plot above you see theraster of 500 excitatory neurons (out of 1000). In the plot below we have the number of spikes counted in bins of 2ms. This is called the population firing rate

- What does this plot tell us?
- Does it mean anything that the average population firing rate is comparable to the average neuron firing rate?
- Can we use the above traces to estimate synchronization in the network?


# <font color=green>Measure of population synchrony</font>
#### Ideally to measure synchrony we should measure cross correlation between each pair of spike trains. This is a bit slow as we need to measure for so many different pairs -- $N^2$. 
#### There is another way to estimate whether the network has synchronized or not?
#### If there are synchronous spikes then a lot of neurons will spike together and go into refractoriness together. That means that synchronization will result in higher than expected variance of the population average. That is, we can estimate the variance of the population activity. However, when we have a Poisson process which by definition is an uncorrelated (not synchronized) population activity variance increases with population activity mean. That is, if firing rate increase we expected higher variance. So we need to normalize the popultion variance by its mean. The ratio variance/mean is called *Fano  Factor* and it is 'one' for a Poisson process. Here we are binning the data so Fano Factor does depend on the bin size. Nevertheless it is a very good measure of syhchrony.

### Lets measure the Fano factor

In [ ]:
FF = np.var(hh)/np.mean(hh)
print(FF)

# <font color=green>Descriptors of Network activity dynamics</font>
#### CV describes the single neuron firing patterns -- when CV is close to zero, neurons spike in a regular (R) manner, when CV is close to one, neurons spike in an irregular (I) manner.
#### FF describes the poopulation firing behavior -- when FF is close to one, population of neurons is near Poissonian and therefore Asynchronous (A), when FF is greater than one, neurons synchronize (S).
## Thus, we can define a network activity as AI, AR, SI and SR
### Of these four states AI and SI are often observed in healthy state. SR and AR usually indicate abnormal dynamics. Too much synchrony is also considered unhealthy.

# <font color=salmon>TO DO</font>
- Vary the network parameters g and eta_ex and study how the network activity state changes.
(In these network when g > 4 we call this as a inhibition dominated regime. The number 4 arises becaue the ratio of exc and inh ratio is 4)
